In [1]:
import numpy as np

# Outils sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Outils pytorch
import torch # package Racine
from torch import nn # Réseau de Neurones (Neural Network)
import torch.nn.functional as F # Couches, fonctions d'activation ...
import torch.autograd as autograd # Calcul dérivée (Gradient)
import torch.optim as optim # Optimiser pour la descente de Gradient

# Librairie graphique plotly

import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

In [2]:
cuda = torch.device('cuda:0')
print('cuda disponible:', torch.cuda.is_available())

dir(optim)

cuda disponible:True


['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'Optimizer',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'lr_scheduler']

## Construction d'une classe module

In [3]:
class ClasseModele(nn.Module):
    
    def __init__(self, input_dim):
        super(ClasseModele, self).__init__()        
        self.layer1 = nn.Linear(input_dim,50) # Première couche de 50 neurones
        self.layer2 = nn.Linear(50, 20)       # deuxième couche de 20 neurones        
        self.layer3 = nn.Linear(20, 3)        # Couche de sortie de 3 neurones (3 classes)
        
    def forward(self, x):
        # https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))        
        x = F.softmax(self.layer3(x),dim=1) # Application de la fonction sofmax à la couche de sortie
        return x    

## Chargement des données Iris

In [4]:
features, labels = load_iris(return_X_y=True)
print(features.shape,labels.shape)
iris = np.concatenate((features, labels.reshape([150,1])), axis=1)
iris[np.random.randint(len(iris), size=10)]

(150, 4)(150,)


array([[6.3, 2.5, 4.9, 1.5, 1. ],
       [6. , 3.4, 4.5, 1.6, 1. ],
       [4.9, 3.1, 1.5, 0.2, 0. ],
       [6.3, 2.5, 4.9, 1.5, 1. ],
       [6.7, 3. , 5.2, 2.3, 2. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [6.7, 3.3, 5.7, 2.5, 2. ],
       [6.5, 3. , 5.2, 2. , 2. ],
       [5.7, 2.6, 3.5, 1. , 1. ]])

## Création d'un set de train et d'un set de test (dev)

In [5]:
features_train,features_dev, labels_train, labels_dev = train_test_split(features, labels, random_state=42)
print(features_train.shape,features_dev.shape, labels_train.shape, labels_dev.shape)
labels_dev[:10]

(112, 4)(38, 4)(112,)(38,)


array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1])

## Apprentissage avec PyTorch

In [6]:
# Création d'un objet Modèle
model = ClasseModele(features_train.shape[1])
model.cuda()

# choix de l'algorithme de Descente de Gradient et du learning Rate
# https://pytorch.org/docs/stable/optim.html#algorithms
optimizer = optim.Adam(model.parameters(), lr=0.01)

# choix de la fonction de coût
# https://pytorch.org/docs/stable/nn.html#loss-functions
loss_fn = nn.CrossEntropyLoss()

#Nombre d'itération sur les données
epochs = 100

print(model)

ClasseModele(
  (layer1): Linear(in_features=4, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=20, bias=True)
  (layer3): Linear(in_features=20, out_features=3, bias=True)
)


In [7]:
x_train, y_train = torch.from_numpy(features_train,).float().cuda(), torch.from_numpy(labels_train).long().cuda()
for epoch in range(1, epochs+1):    
    y_pred = model(x_train)
    loss = loss_fn(y_pred, y_train)
    if not epoch%10 :        
        print('Epoch #%i Loss=%.4f'%(epoch,loss))               
    
    optimizer.zero_grad() # Réinitialise le gradient
    loss.backward()       # Exécute la backpropagation
    optimizer.step()      # Met à jours les paramètres du réseau

Epoch #10 Loss=0.8617
Epoch #20 Loss=0.7123
Epoch #30 Loss=0.6125
Epoch #40 Loss=0.5872
Epoch #50 Loss=0.5808
Epoch #60 Loss=0.5783
Epoch #70 Loss=0.5771
Epoch #80 Loss=0.5762
Epoch #90 Loss=0.5756
Epoch #100 Loss=0.5751


## Prédiction

In [8]:
#Création du Tensor x_dev, de type float
x_dev = torch.from_numpy(features_dev).float().cuda()
#Exécution du modèle
pred = model(x_dev)
#Sortie pred du computation graph, et conversion en tableau numpy
print(pred.grad_fn)
pred = pred.detach()
print(pred.grad_fn)
pred = pred.cpu().numpy()
pred

None


array([[2.11814586e-06, 9.99039829e-01, 9.57994838e-04],
       [9.99898672e-01, 1.01308884e-04, 1.92183415e-30],
       [2.59754196e-25, 6.41509036e-13, 1.00000000e+00],
       [2.00324871e-06, 9.97657299e-01, 2.34068790e-03],
       [6.31911462e-07, 9.99325633e-01, 6.73766132e-04],
       [9.99789536e-01, 2.10450569e-04, 3.90722556e-28],
       [7.42901000e-04, 9.99257028e-01, 2.71897260e-08],
       [4.47798812e-11, 6.06434094e-03, 9.93935645e-01],
       [1.38356295e-08, 1.04787119e-01, 8.95212829e-01],
       [1.03872917e-04, 9.99895096e-01, 1.05324591e-06],
       [4.55052707e-09, 1.15273111e-01, 8.84726882e-01],
       [9.99618649e-01, 3.81323538e-04, 3.71942956e-26],
       [9.99889255e-01, 1.10676789e-04, 3.33586150e-30],
       [9.99663234e-01, 3.36778117e-04, 1.50291515e-26],
       [9.99844790e-01, 1.55154019e-04, 4.55161343e-29],
       [2.72710236e-06, 9.99891520e-01, 1.05793246e-04],
       [7.22757580e-18, 3.18486748e-08, 1.00000000e+00],
       [6.14606834e-05, 9.99927

## Calcul du score de précision

### Transformation des probabilités en résultat unique

In [9]:
prevision_type_iris =  np.argmax(model(x_dev).cpu().detach().numpy(),axis=1)
prevision_type_iris

array([1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0], dtype=int64)

### Calcul de la précision

In [10]:
print ("Précision de la prévision = %.1f%% "%(accuracy_score(labels_dev, prevision_type_iris)*100))
print('%i prévision(s) correcte(s), %i erreur(s)'%(sum(prevision_type_iris==labels_dev),sum(prevision_type_iris!=labels_dev)))

Précision de la prévision = 97.4%
37 prévision(s) correcte(s), 1 erreur(s)


### Comparaison Courbes 

In [11]:
acc_train=[]
acc_dev=[]
tab_epoch=[]
new_model = ClasseModele(features_train.shape[1])
new_model.cuda()
new_optimizer = optim.Adam(new_model.parameters(), lr=0.01)
epocs=100
for epoch in range(1, epochs+1):    
    y_pred = new_model(x_train)
    loss = loss_fn(y_pred, y_train)
    
    if not epoch%10 :                
        print('Epoch #%i Loss=%.2f'%(epoch,loss.item()))        
        tab_epoch.append(epoch)
        acc_dev.append(
            accuracy_score(
                labels_dev,
                np.argmax(new_model(x_dev).cpu().detach().numpy(),axis=1)
            )*100            
        )
        acc_train.append(
            accuracy_score(
                labels_train,
                np.argmax(new_model(x_train).cpu().detach().numpy(),axis=1)
            )*100
        )           
    new_optimizer.zero_grad() # Réinitialise le gradient
    loss.backward()       # Exécute la backpropagation
    new_optimizer.step()      # Met à jours les paramètres du réseau

print(acc_train,acc_dev)

Epoch #10 Loss=0.96
Epoch #20 Loss=0.77
Epoch #30 Loss=0.64
Epoch #40 Loss=0.60
Epoch #50 Loss=0.58
Epoch #60 Loss=0.58
Epoch #70 Loss=0.58
Epoch #80 Loss=0.58
Epoch #90 Loss=0.58
Epoch #100 Loss=0.58
[65.17857142857143, 84.82142857142857, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571][71.05263157894737, 84.21052631578947, 100.0, 97.36842105263158, 97.36842105263158, 97.36842105263158, 97.36842105263158, 97.36842105263158, 97.36842105263158, 97.36842105263158]


## Graphique

In [12]:
train_line =go.Scatter(x=tab_epoch,y=acc_train,name='Train set')
dev_line =go.Scatter(x=tab_epoch,y=acc_dev,name='Dev set')

layout = go.Layout(title="Comparaison Précision Train/Test",titlefont=dict(size=40),autosize=False, width=1100,height=1100)

data=[train_line,dev_line]
fig = go.Figure(data=data, layout=layout)
fig.show()

In [13]:
# import plotly.graph_objects as go
# import numpy as np

x = np.arange(10)

fig = go.Figure(data=go.Scatter(x=x, y=x**2))
fig.show()